In [1]:
import joblib
import pandas as pd

FEATURES = [
    "duration",
    "credit_amount",
    "age",
    "checking_status",
    "employment",
    "savings_status",
    "purpose",
]

model = joblib.load("../artifacts/models/model_v1.joblib")

sample = {
    "duration": 24,
    "credit_amount": 4500,
    "age": 28,
    "checking_status": "no checking",
    "employment": "unemployed",
    "savings_status": "<100",
    "purpose": "vacation/others"
}

X_new = pd.DataFrame([sample], columns=FEATURES)

proba_good = float(model.predict_proba(X_new)[:, 1][0])
decision = "APROBADO" if proba_good >= 0.5 else "RECHAZADO"

print({"proba_good": round(proba_good, 4), "decision": decision})


{'proba_good': 0.7675, 'decision': 'APROBADO'}


# Prediction on New Instances (Model Serving)

This notebook demonstrates prediction on new client samples using the trained credit risk model.

A prediction function is defined that:

- receives new client data
- applies preprocessing pipeline
- generates probability prediction
- outputs predicted class

Multiple prediction examples are executed to validate inference behavior.


## Prediction Function Definition

This function encapsulates preprocessing and model inference steps to ensure consistent prediction behavior.
